In [1]:
# day9 - https://adventofcode.com/2019/day/9

In [2]:
# this object is where the Intcode program lives and some useful methods
# to fill it, clear it, read from it, write to it
class Computer:
    import logging
    global set_trace
    global inputs
    global outputs
    
    def __init__(self, size, stage, rbase=0):
        # size is array size[0] = program memory, size[1] = general memory
        self.size = size  # must specify the memory size
        self.stage = stage
        self.storage = [None] * (size[0] + size[1])  # initialize with None
        # initial relative_base
        self.rbase = rbase
        # program base memory is 0
        self.prog_base = 0
        # memory space is immediately above the programming space
        self.mem_base = size[0] + 1
    
    # set the releative_base value
    def set_rbase(self, val):
        self.rbase = val
        return
        
    # dump out the contents between 2 positions (defaults all the memory)
    def dump(self,start=0,end=-1):
        if(end == -1):
            end = self.size
        for i in range(start, end):
            print("{}: {}".format(i,self.storage[i]))

    # fill the memory with a sequence, starting at a certain position - which defaults to 0
    def fill(self, codelist, start_pos = 0):
        i = start_pos
        for j in codelist:
            self.storage[i] = j
            i+=1
    
    # fill (write to) memory with a sequence, allowing direct or reference starting location
    def mode_fill(self, s, mem_location, mode):
        # mem_location: where to begin fill or to reference to begin
        # mode (0 - position, 1 - immediate)
        if(set_trace): import pdb; pdb.set_trace()
        if(mode == '0' or mode == '2'):
            # these are positional references
            i = self.storage[self.relative_location(mem_location, mode)]
            for j in s:
                self.storage[i] = j
                i+=1
        elif(mode == '1'):
            # this is the direct reference
            i = mem_location
            for j in s:
                self.storage[i] = j
                i+=1

    # load the program into program space
    def load_program(self, s):
        self.mode_fill(s, self.prog_base, "1")
        return
                
    # get data from input_buffer for this stage
    # if it has 2 values, then it hasn't read in the amp_setting
    # that state will return the last value and remove it

#     def getfrominput(self):
#         if(len(inputs[self.stage].contents) > 1):
#             # pop a value off and return it
#             return(inputs[self.stage].contents.pop())
#         else:
#             # return just that last value but leave it
#             return(inputs[self.stage].contents[0])

    # get data from single input buffer
    def getfrominput(self):
        return(inputs)

    # put data to output_buffer for this stage (which is next stage's input)
    # put this output value into position [0] of that buffer (position [1] will have possible amp_setting)
#     def puttooutput(self, val):
#         logging.info("outputting: {} to stage: {}".format(val, next_stage(self.stage,5)))
#         inputs[next_stage(self.stage,5)].contents[0] = val
#         return

    # output to single output buffer
    def puttooutput(self, val):
        global outputs
        logging.info("before put - outputs: {}".format(outputs))
        outputs = val
        logging.info("after put - outputs: {}".format(outputs))
        return

    # read the contents from just one position
    def read(self, pos):
        return self.storage[pos]
   
    # return the value based on parameter mode
    def mode_read(self, pos, mode):
        if(set_trace): import pdb; pdb.set_trace()
        # mode: 0 - position, 1 - immediate, 2 - relative
        if(mode == '0' or mode == '2'):
            return int(self.storage[self.storage[self.relative_location(pos, mode)]])
        elif(mode == '1'):
            return int(self.storage[pos])

    # clear the memory
    def clear(self):
        self.storage = [None] * self.size

    # return the location adjusted by relative_base, based on mode
    def relative_location(self, location, mode):
        # location - input location
        # mode - the parameter mode -'0': position, '2': relative
        return((location + self.rbase) if int(mode) else location)
        
    # this function will execute a single sequence (operator + subsequent operands and output location)
    def exec_seq(self, ctr):
        global set_trace
        # these string representations of opcodes are used in logging
        opcodes = {"99":"STOP","01":"ADD","02":"MULTIPLY","03":"INPUT","04":"OUTPUT","05":"JUMP-TRUE","06":"JUMP-FALSE","07":"LESS-THAN","08":"EQUALS-TO", "09":"ADJUST-RBASE"}
        # ctr - where to start
        # m - the memory object to work from
        # the first in the sequence is the 'enhnanced opcode' (includes 3 parameter modes in most significant position)
        # zero-pad to 5 - most significant 0 is implied
        enhanced_opcode = str(self.read(ctr)).zfill(5)
        # first 3 characters are the parameter modes
        pmodes = enhanced_opcode[:3]
        # last 2 characters are the opcode
        opcode = enhanced_opcode[3:]
        if(set_trace): import pdb; pdb.set_trace()
        # if it is 99, we're done
        if(opcode == '99'):
            prog_status[self.stage] = "STOP"
            logging.info("opcode: {} - {}".format(opcode, opcodes[opcode]))
            return("STOP")
        # this is the "add" operator
        elif(opcode == "01"):
            if(set_trace): import pdb; pdb.set_trace()
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            if(set_trace): import pdb; pdb.set_trace()
            calcval = operand1 + operand2
        # this is the "multiply" operator
        elif(opcode == "02"):
            if(set_trace): import pdb; pdb.set_trace()
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            calcval = operand1 * operand2
        # this is the 'store from input' operator
        elif(opcode == "03"):
            if(set_trace): import pdb; pdb.set_trace()
            # get the input
            # val = getfrominput()
            # the input buffer is the one for this amp stage
            val = self.getfrominput()
            # store it in address provided by next code
            self.mode_fill([val], ctr+1, pmodes[2])
            # return next prog counter
            logging.info("opcode: {}-{}, pmodes: {}, from-input: {}, @ctr+1: {}, ctr in: {}, returning: {}".format(opcode,opcodes[opcode],pmodes,val,self.read(ctr+1),ctr,ctr+2))
            return(ctr+2)
        # this is the 'send to output' operator
        elif(opcode == "04"):
            if(set_trace): import pdb; pdb.set_trace()
            # read the val (based on mode) specified from next code and send to output
            self.puttooutput(self.mode_read(ctr+1, pmodes[2]))
            prog_status[self.stage] = "OUTPUT"
            logging.info("opcode: {}-{}, pmodes: {}, @ctr+1: {}, ctr in: {}, returning: {}".format(opcode, opcodes[opcode],pmodes,self.read(ctr+1),ctr,ctr+2))
            return(ctr+2)
        # this is 'jump if true'
        elif(opcode == "05"):
            val_to_test = self.mode_read(ctr+1, pmodes[2])
            if(val_to_test):
                ret_ctr = self.mode_read(ctr+2, pmodes[1])
            else:
                ret_ctr = ctr+3
            logging.info("opcode: {}-{}, pmodes: {}, ctr+1: {}, val_to_test: {}, ctr+2: {}, ctr+3: {}, ctr in: {}, returning: {}".format(opcode, opcodes[opcode],pmodes,self.read(ctr+1),val_to_test, self.read(ctr+2), self.read(ctr+3), ctr, ret_ctr))
            return(ret_ctr)
        # this is 'jump if false'
        elif(opcode == "06"):
            val_to_test = self.mode_read(ctr+1, pmodes[2])
            if(val_to_test):
                ret_ctr = ctr+3
            else:
                ret_ctr = self.mode_read(ctr+2, pmodes[1])
            logging.info("opcode: {}-{}, pmodes: {}, ctr+1: {}, val_to_test: {}, ctr+2: {}, ctr+3: {}, ctr in: {}, returning: {}".format(opcode, opcodes[opcode],pmodes,self.read(ctr+1),val_to_test, self.read(ctr+2), self.read(ctr+3), ctr, ret_ctr))
            return(ret_ctr)
        # this is 'less than' operator
        elif(opcode == "07"):
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            if(operand1 < operand2):
                calcval = 1
            else:
                calcval = 0
        # this is the 'equals to' operator
        elif(opcode == "08"):
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            if(operand1 == operand2):
                calcval = 1
            else:
                calcval = 0
        # this adjusts the relative_base
        elif(opcode == "09"):
            # modify rbase by the value in next position
            self.rbase = self.rbase + self.mode_read(ctr+1, pmodes[2])
            logging.info("opcode: {}-{}, pmodes: {}, ctr+1: {}, self.rbase: {}, ctr in: {}, returning: {}".format(opcode, opcodes[opcode],pmodes,self.read(ctr+1),self.rbase,ctr,ctr+2))
            return(ctr+2)
        # if here, then the opcode is not known
        else:
            prog_status[self.stage] = "BADOPCODE"
            logging.info("opcode: {}-{}".format(opcode,opcodes[opcode]))
            return("BADOPCODE")
        # fill the output destination with value depending on mode
        if(set_trace): import pdb; pdb.set_trace()
        self.mode_fill([calcval], ctr+3, pmodes[0])
        logging.info("opcode: {}-{}, pmodes: {}, ctr+1: {}, op1: {}, ctr+2: {}, op2: {}, calcval: {}, ctr+3: {}, ctr in: {}, returning: {}".format(opcode, opcodes[opcode],pmodes,self.read(ctr+1),operand1,self.read(ctr+2),operand2,calcval,self.read(ctr+3),ctr,ctr+4))
        # and return the next postion in the memory
        return(ctr+4)

In [3]:
inputs = None
outputs = None
prog_status = [None] * 1

In [4]:
# run parameters
# seq = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
run_name = "day9"
seq = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
inputs = 0
prog_ctr = 0
set_trace = False
set_trace_main = False

In [5]:
# this is used for logging
import time
year, month, day, hour, minute = time.strftime("%Y %m %d %H %M").split()
# log file name
lfn = "{}-{}{}{}{}{}.log".format(run_name,year,month,day,hour,minute)
# set up the logging
import logging
logging.basicConfig(filename=lfn, level=logging.INFO)
logging.info("here we go....")

In [6]:
# initialize computer
# size array is [prog_space, memory_space]
comp = Computer([len(seq),200], 0)

In [7]:
# load the program
comp.load_program(seq)

In [8]:
# run it
while(prog_ctr != "STOP"):
    if(set_trace_main): import pdb; pdb.set_trace()
    prog_ctr = comp.exec_seq(prog_ctr)

IndexError: list index out of range

In [ ]:
# print the prog_status and the output
print(prog_status[0])
print(outputs)